In [54]:
import geopandas as gpd
from datetime import datetime
from libcomcat.search import search
import pandas as pd
import pprint

# Import

In [2]:

# =============================================================================
# =============================================================================

# --- 1. Paramètres utilisateur ---
geojson_path = "myanmar.json"    # <- à adapter
date_start = "2025-03-22"   # au format YYYY-MM-DD
date_end   = "2025-04-30"
# =============================================================================
# =============================================================================



# --- 2. Charger le GeoJSON et s'assurer du bon CRS ---
gdf = gpd.read_file(geojson_path)
if gdf.crs is not None and gdf.crs.to_epsg() != 4326:
    gdf = gdf.to_crs(4326)
minx, miny, maxx, maxy = gdf.total_bounds

# --- 3. Conversion des dates ---
starttime = datetime.fromisoformat(date_start)
endtime = datetime.fromisoformat(date_end)

# --- 4. Recherche des événements ---
events = search(
    minlongitude=minx,
    minlatitude=miny,
    maxlongitude=maxx,
    maxlatitude=maxy,
    starttime=starttime,
    endtime=endtime,
)

# --- 5. Afficher les attributs du premier événement pour inspecter ---
if events:
    first = events[0]
    print(first)
    print("----------- Attributs disponibles ----------")
    print(dir(first))
else:
    print("Aucun événement trouvé.")

# --- 6. Construire le DataFrame selon les attributs disponibles ---
# Liste possible des attributs à récupérer
cols = ["id", "time", "magnitude", "depth", "latitude", "longitude", "magtype", "gap", "rms", "horizontal_error", "vertical_error"]

data = []
for e in events:
    d = {}
    for c in cols:
        d[c] = getattr(e, c, None)
    data.append(d)
df = pd.DataFrame(data)
print(df.head())

# Tu pourras adapter la liste 'cols' en fonction de ce que tu vois dans le dir(first)


us6000q3p6 2025-03-24 04:38:32.700000 (29.189,93.747) 10.0 km M4.2
----------- Attributs disponibles ----------
['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_jdict', 'alert', 'depth', 'getDetailEvent', 'getDetailURL', 'hasProduct', 'hasProperty', 'id', 'latitude', 'location', 'longitude', 'magnitude', 'properties', 'time', 'toDict', 'url']
           id                             time  magnitude  depth  latitude  \
0  us6000q3p6 2025-03-24 04:38:32.700000+00:00        4.2   10.0   29.1888   
1  us6000q3ny 2025-03-24 04:43:00.743000+00:00        4.1   10.0   29.2119   
2  us7000pm7w 2025-03-24 11:21:38.311000+00:00        5.0   10.0   28.1045   
3  us7000pmg7 

In [3]:
df = df.sort_values("magnitude", ascending=False)
df

,id,time,magnitude,depth,latitude,longitude,magtype,gap,rms,horizontal_error,vertical_error
5,us7000pn9s,2025-03-28 06:20:52.715000+00:00,7.7,10.0,22.0110,95.9363,None,None,None,None,None
6,us7000pn9z,2025-03-28 06:32:04.777000+00:00,6.7,10.0,21.6975,95.9690,None,None,None,None,None
66,us6000q5ps,2025-04-13 02:24:57.717000+00:00,5.3,10.0,21.2292,96.0897,None,None,None,None,None
78,us7000pv6f,2025-04-26 04:21:49.088000+00:00,5.1,10.0,31.0900,98.9820,None,None,None,None,None
30,us7000pnkj,2025-03-29 09:20:48.229000+00:00,5.1,10.0,19.6858,96.0761,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...
64,us6000q5cs,2025-04-11 10:52:56.066000+00:00,4.1,10.0,24.0430,91.3615,None,None,None,None,None
59,us6000q575,2025-04-10 16:17:58.886000+00:00,3.9,10.0,21.7193,95.9475,None,None,None,None,None
56,us6000q4x4,2025-04-09 14:40:17.442000+00:00,3.6,10.0,21.4447,95.9352,None,None,None,None,None
57,us6000q4xc,2025-04-09 15:59:18.014000+00:00,3.6,10.0,21.9457,96.0058,None,None,None,None,None


## Création d'un geo df

In [4]:

import geopandas as gpd
from shapely.geometry import Point

gdf_map = gpd.GeoDataFrame(
    df,
    geometry=[Point(xy) for xy in zip(df.longitude, df.latitude)],
    crs="EPSG:4326"
)


## Check sur une carte

In [5]:
# Load a map with data and config and height
from keplergl import KeplerGl
map_1 = KeplerGl(height=1000, show_docs=False)
map_1.add_data(data=df, name="Séismes")
map_1


# map_1.save_to_html(file_name="data_output/seismes_map.html")


KeplerGl(data={'Séismes': {'index': [5, 6, 66, 78, 30, 2, 77, 48, 8, 9, 7, 44, 61, 31, 43, 76, 68, 81, 47, 35,…

In [6]:
import folium
import numpy as np  # pour np.exp

# Exemple : Rayon exponentiel en fonction de la magnitude
def mag_to_radius(mag):
    # Version exponentielle "douce" :
    return 2 * np.exp(mag - 4)  # ajuster le facteur si besoin


# Création de la carte
m = folium.Map(location=[df["latitude"].mean(), df["longitude"].mean()], zoom_start=6, tiles="cartodbpositron")

for _, row in df.iterrows():
    radius = mag_to_radius(row["magnitude"]) if row["magnitude"] else 2
    folium.CircleMarker(
        location=[row["latitude"], row["longitude"]],
        radius=radius,
        popup=f"Mag {row['magnitude']}<br>ID: {row['id']}",
        color="#3186cc",
        fill=True,
        fill_opacity=0.6
    ).add_to(m)

m  # affichage dans le notebook


In [38]:
event_id = df.iloc[0]["id"]
print(event_id)  # Ex: 'us6000kikc'


us7000pn9s


### Accès à toutes les données de Shakemap

In [55]:
import requests
import json

event_id = "us7000pn9s"
url = f"https://earthquake.usgs.gov/fdsnws/event/1/query?eventid={event_id}&format=geojson"
r = requests.get(url)
data = r.json()
data

{'type': 'Feature',
 'properties': {'mag': 7.7,
  'place': '2025 Mandalay, Burma (Myanmar) Earthquake',
  'time': 1743142852715,
  'updated': 1751002490930,
  'tz': None,
  'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/us7000pn9s',
  'felt': 2526,
  'cdi': 9.1,
  'mmi': 9.953,
  'alert': 'red',
  'status': 'reviewed',
  'tsunami': 0,
  'sig': 2910,
  'net': 'us',
  'code': '7000pn9s',
  'ids': ',us7000pn9s,usauto7000pn9s,pt25087002,at00sttln4,',
  'sources': ',us,usauto,pt,at,',
  'types': ',dyfi,earthquake-name,finite-fault,general-text,ground-failure,impact-text,internal-moment-tensor,internal-origin,losspager,moment-tensor,oaf,origin,phase-data,shakemap,',
  'nst': 246,
  'dmin': 3.474,
  'rms': 0.89,
  'gap': 24,
  'magType': 'mww',
  'type': 'earthquake',
  'title': 'M 7.7 - 2025 Mandalay, Burma (Myanmar) Earthquake',
  'products': {'dyfi': [{'indexid': '26627303',
     'indexTime': 1751002496395,
     'id': 'urn:usgs-product:us:dyfi:us7000pn9s:1751002490930',
     'ty

In [62]:
from datetime import datetime

files = []

for product_type, product_list in products.items():
    for product in product_list:
        contents = product.get('contents', {})
        if isinstance(contents, dict):
            for file_name, file_info in contents.items():
                # Récupérer le timestamp s'il existe
                lastmod = file_info.get("lastModified")
                # Conversion si présent
                if lastmod is not None:
                    # lastmod est en millisecondes
                    lastmod_str = datetime.utcfromtimestamp(int(lastmod) / 1000).strftime('%Y-%m-%d %H:%M:%S')
                else:
                    lastmod_str = "unknown"
                files.append({
                    "product_type": product_type,
                    "file_name": file_name,
                    "url": file_info.get("url"),
                    "size": file_info.get("length", "unknown"),
                    "content_type": file_info.get("contentType", "unknown"),
                    "last_modified": lastmod_str
                })

files = sorted(files, key=lambda x: (x['product_type'], x['file_name']))

# Printing products

for f in files:
    print(
        f"{f['product_type']:16} {f['file_name']:40} "
        f"{f['size']:>10} bytes  {f['last_modified']:19}  {f['url']}"
    )

products_listing = []

for f in files:
    products_listing.append({
        "product_type": f["product_type"],
        "file_name": f["file_name"],
        "size": f["size"],
        "last_modified": f["last_modified"],
        "url": f["url"]
    })

df_product = pd.DataFrame(products_listing)



dyfi             cdi_geo.txt                                   33462 bytes  2025-06-27 05:17:03  https://earthquake.usgs.gov/product/dyfi/us7000pn9s/us/1751001424084/cdi_geo.txt
dyfi             cdi_geo.xml                                  110595 bytes  2025-06-27 05:17:03  https://earthquake.usgs.gov/product/dyfi/us7000pn9s/us/1751001424084/cdi_geo.xml
dyfi             cdi_geo_1km.txt                              115151 bytes  2025-06-27 05:17:03  https://earthquake.usgs.gov/product/dyfi/us7000pn9s/us/1751001424084/cdi_geo_1km.txt
dyfi             cdi_zip.txt                                   23085 bytes  2025-06-27 05:17:03  https://earthquake.usgs.gov/product/dyfi/us7000pn9s/us/1751001424084/cdi_zip.txt
dyfi             cdi_zip.xml                                   48598 bytes  2025-06-27 05:17:03  https://earthquake.usgs.gov/product/dyfi/us7000pn9s/us/1751001424084/cdi_zip.xml
dyfi             contents.xml                                   4224 bytes  2025-06-27 05:17:03  https://e

/var/folders/8b/jbf_9gw94xs45f1kt0j8k5880000gq/T/ipykernel_32708/3745355469.py:15: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  lastmod_str = datetime.utcfromtimestamp(int(lastmod) / 1000).strftime('%Y-%m-%d %H:%M:%S')


In [65]:
df_product.to_csv(f"data_output/products_id_{event_id}.csv")

In [95]:
df_product

,product_type,file_name,size,last_modified,url
0,dyfi,cdi_geo.txt,33462,2025-06-27 05:17:03,https://earthquake.usgs.gov/product/dyfi/us700...
1,dyfi,cdi_geo.xml,110595,2025-06-27 05:17:03,https://earthquake.usgs.gov/product/dyfi/us700...
2,dyfi,cdi_geo_1km.txt,115151,2025-06-27 05:17:03,https://earthquake.usgs.gov/product/dyfi/us700...
3,dyfi,cdi_zip.txt,23085,2025-06-27 05:17:03,https://earthquake.usgs.gov/product/dyfi/us700...
4,dyfi,cdi_zip.xml,48598,2025-06-27 05:17:03,https://earthquake.usgs.gov/product/dyfi/us700...
...,...,...,...,...,...
176,shakemap,download/shake_result.hdf,31963388,2025-06-06 13:36:22,https://earthquake.usgs.gov/product/shakemap/u...
177,shakemap,download/shakemap.kmz,831540,2025-06-06 13:36:22,https://earthquake.usgs.gov/product/shakemap/u...
178,shakemap,download/shape.zip,26264669,2025-06-06 13:36:22,https://earthquake.usgs.gov/product/shakemap/u...
179,shakemap,download/stationlist.json,3499574,2025-06-06 13:36:22,https://earthquake.usgs.gov/product/shakemap/u...


## Get MMI shapefiles

In [79]:
import os
import requests
import zipfile
import geopandas as gpd
from glob import glob

# --- Paramètres ---
mmi_url = df_product[df_product["file_name"] == "download/shape.zip"]["url"].iloc[0]
base_dir = "data_output/shape_mmi"
os.makedirs(base_dir, exist_ok=True)
zip_path = os.path.join(base_dir, "mmi.zip")

# 1. Télécharger le ZIP
with requests.get(mmi_url, stream=True) as r:
    r.raise_for_status()
    with open(zip_path, "wb") as f:
        for chunk in r.iter_content(chunk_size=8192):
            f.write(chunk)

# 2. Extraire uniquement les .shp
with zipfile.ZipFile(zip_path, "r") as zip_ref:
    shp_members = [name for name in zip_ref.namelist() if name.endswith(".shp")]
    zip_ref.extractall(
        base_dir,
        members=shp_members
    )

# 3. Supprimer le ZIP
os.remove(zip_path)

# 4. Lister les .shp extraits
shp_files = []
for root, dirs, files in os.walk(base_dir):
    for file in files:
        if file.endswith(".shp"):
            rel_path = os.path.relpath(os.path.join(root, file), base_dir)
            shp_files.append(rel_path)

print("Shapefiles extraits :")
for path in shp_files:
    print("-", path)


Shapefiles extraits :
- psa3p0.shp
- psa1p0.shp
- psa0p3.shp
- pgv.shp
- pga.shp
- mi.shp


In [81]:

shp_path = os.path.join(base_dir, "mi.shp")

# Lecture du shapefile (les fichiers .shx, .dbf, .prj doivent être présents au même endroit)
gdf = gpd.read_file(shp_path)

# Aperçu attributs
print(gdf.head())

# Colonnes disponibles
print(gdf.columns)

# Exemple d'affichage d'une géométrie
print(gdf.geometry.iloc[0])

   AREA  PERIMETER  PGAPOL_  PGAPOL_ID  GRID_CODE  PARAMVALUE  \
0   NaN        NaN       14         14          0         2.8   
1   NaN        NaN       15         15          0         3.0   
2   NaN        NaN       16         16          0         3.2   
3   NaN        NaN       17         17          0         3.4   
4   NaN        NaN       18         18          0         3.6   

                                            geometry  
0  MULTIPOLYGON (((102.15192 26.75, 102.15032 26....  
1  MULTIPOLYGON (((101.22624 26.775, 101.22608 26...  
2  MULTIPOLYGON (((100.12571 26.775, 100.12539 26...  
3  MULTIPOLYGON (((97.07814 26.725, 97.07574 26.7...  
4  MULTIPOLYGON (((97.60229 26.725, 97.60131 26.7...  
Index(['AREA', 'PERIMETER', 'PGAPOL_', 'PGAPOL_ID', 'GRID_CODE', 'PARAMVALUE',
       'geometry'],
      dtype='object')
MULTIPOLYGON (((102.151915 26.75, 102.150315 26.749685, 102.15 26.746747, 102.149599 26.749599, 102.149697 26.75, 102.149849 26.750151, 102.15 26.75075, 102.1

In [88]:
gdf["PARAMVALUE_CLASS"] = gdf["PARAMVALUE"].astype(int)
import numpy as np
gdf["PARAMVALUE_CLASS"] = np.clip(gdf["PARAMVALUE"].astype(int), 0, int(gdf["PARAMVALUE"].max()))
gdf_dissolved = gdf.dissolve(by="PARAMVALUE_CLASS")


In [94]:
# gdf.explore(column="PARAMVALUE", cmap="OrRd", tiles="OpenStreetMap")
gdf_dissolved.explore(column="PARAMVALUE", cmap="OrRd", tiles="OpenStreetMap")


In [92]:
gdf_dissolved

,geometry,AREA,PERIMETER,PGAPOL_,PGAPOL_ID,GRID_CODE,PARAMVALUE
PARAMVALUE_CLASS,,,,,,,
2,"MULTIPOLYGON (((102.13081 13.03081, 102.125 13...",NaN,NaN,14,14,0,2.8
3,"MULTIPOLYGON (((90.4 16.40328, 90.4 21.87164, ...",NaN,NaN,15,15,0,3.0
4,"MULTIPOLYGON (((91.9 21.48347, 91.90848 21.483...",NaN,NaN,20,20,0,4.0
5,"MULTIPOLYGON (((93.65579 18.80579, 93.65 18.80...",NaN,NaN,25,25,0,5.0
6,"MULTIPOLYGON (((95.025 17.11756, 95.01865 17.1...",NaN,NaN,30,30,0,6.0
7,"MULTIPOLYGON (((95.78991 17.63509, 95.79219 17...",NaN,NaN,35,35,0,7.0
8,"MULTIPOLYGON (((96.53914 17.98914, 96.5349 17....",NaN,NaN,40,40,0,8.0
9,"POLYGON ((96.44852 18.19852, 96.44723 18.2, 96...",NaN,NaN,45,45,0,9.0
10,"POLYGON ((95.975 22.36188, 95.97259 22.37259, ...",NaN,NaN,50,50,0,10.0


## Access to product - function

In [ ]:
def get_json_url(df, pattern):
    """Retourne l'URL du premier fichier .json/.geojson contenant le pattern."""
    mask = df['file_name'].str.contains(pattern, case=False, na=False) & (
        df['file_name'].str.endswith('.json') | df['file_name'].str.endswith('.geojson')
    )
    results = df[mask]
    if results.empty:
        raise ValueError(f"No file matching '{pattern}' found.")
    return results.iloc[0]['url'], results.iloc[0]['file_name']

### Cities

In [109]:
df_cities = get_parsed_json_resource(df_product, "cities.json", key="all_cities")
df_cities.columns

Index(['name', 'ccode', 'lat', 'lon', 'iscap', 'pop', 'mmi', 'on_map'], dtype='object')

### Exposure

In [110]:
def parse_exposure_json(data):
    pop = data["population_exposure"]
    econ = data["economic_exposure"]
    mmis = pop["mmi"]

    records = []
    for c_pop, c_econ in zip(pop["country_exposures"], econ["country_exposures"]):
        country_code = c_pop["country_code"]
        for mmi_val, p_exp, e_exp in zip(mmis, c_pop["exposure"], c_econ["exposure"]):
            records.append({
                "country_code": country_code,
                "mmi": mmi_val,
                "population_exposure": p_exp,
                "economic_exposure": e_exp
            })
    return pd.DataFrame(records)

def get_parsed_resource(df_resources, filename_contains, parser, key=None):
    """
    Récupère une ressource JSON et la parse selon la fonction `parser`.

    Args:
      - df_resources : DataFrame avec colonnes 'file_name' et 'url'
      - filename_contains : str fragment à chercher dans 'file_name'
      - parser : fonction qui prend en entrée le JSON (dict) et renvoie un DataFrame
      - key : optionnel, clé à extraire dans le JSON avant parsing (utile pour villes par ex.)

    Returns:
      - DataFrame parsé
    """
    url = find_resource_url(df_resources, filename_contains)
    print(f"🔗 Téléchargement depuis : {url}")
    response = requests.get(url)
    response.raise_for_status()
    data = response.json()

    if key is not None:
        if key not in data:
            raise KeyError(f"Clé '{key}' non trouvée dans le JSON.")
        data_to_parse = data[key]
    else:
        data_to_parse = data

    return parser(data_to_parse)

In [ ]:
df_exposure = get_parsed_json_resource(df_product, "exposure.json", key="all_cities")

In [111]:
df_exposure = get_parsed_resource(
    df_product,
    filename_contains="exposure",
    parser=parse_exposure_json,
    key=None
)

🔗 Téléchargement depuis : https://earthquake.usgs.gov/product/losspager/us7000pn9s/us/1744780945901/exposure.pdf


JSONDecodeError: Expecting value: line 1 column 1 (char 0)